# 연예인 분류 모델 만들기

## 1. 데이터 수집하기

In [ ]:
# 이미지 저장할 저장소 만들기
from pathlib import Path 

folder_name = "celebrity"
subfolders = ["train", "val", "test"]

folder = Path(folder_name)
for dir in subfolders:
    dir_path = folder / dir
    if not dir_path.exists():
        dir_path.mkdir(parents=True, exist_ok=True)

In [ ]:
# 데이터 수집 - 라이브러리 설치 pip install bing_image_downloader
## 1개 이미지 다운로드 테스트
from bing_image_downloader import downloader

downloader.download(
    "카리나",
    limit=1,
    output_dir="./celebrity",
    adult_filter_off=True,
    force_replace=False,
    timeout=60
)

[%] Downloading Images to c:\wanted\celebrity\카리나


[!!]Indexing page: 1

[%] Indexed 1 Images on Page 1.


[%] Downloading Image #1 from https://img.hankyung.com/photo/202402/BF.35878118.1.jpg
[%] File Downloaded !



[%] Done. Downloaded 1 images.


In [5]:
from bing_image_downloader import downloader

celeb_list = ["카리나", "마동석", "유재석"]

for celeb in celeb_list:
    downloader.download(
        celeb,
        limit=100,
        output_dir="./celebrity",
        adult_filter_off=True,
        force_replace=False,
        timeout=60
    )

[%] Downloading Images to c:\wanted\celebrity\카리나


[!!]Indexing page: 1

[%] Indexed 95 Images on Page 1.


[%] Downloading Image #1 from https://img.hankyung.com/photo/202402/BF.35878118.1.jpg
[%] File Downloaded !

[%] Downloading Image #2 from https://pbs.twimg.com/profile_images/1433463773856079881/34gC8TAh_400x400.jpg
[%] File Downloaded !

[%] Downloading Image #3 from https://i.ytimg.com/vi/tZixREYOIZQ/maxresdefault.jpg
[%] File Downloaded !

[%] Downloading Image #4 from https://i.ytimg.com/vi/kpEbQOzJb00/maxresdefault.jpg
[%] File Downloaded !

[%] Downloading Image #5 from https://i.ytimg.com/vi/5qcA6xKRcn8/maxresdefault.jpg
[%] File Downloaded !

[%] Downloading Image #6 from https://i.pinimg.com/236x/09/63/7b/09637bdc3f371e6ceeeef0c5577b2be5.jpg
[%] File Downloaded !

[%] Downloading Image #7 from https://i.ytimg.com/vi/yWm4PtbdxUA/maxresdefault.jpg
[%] File Downloaded !

[%] Downloading Image #8 from https://i.ytimg.com/vi/e1FlHbGUw9I/maxresdefault.jpg
[%] File Downloaded

In [9]:
test = "C:/wanted/celebrity/마동석/Image_10.webp"
from PIL import Image 
import numpy as np

image = Image.open(test).convert("RGB")

np.array(image).shape

(571, 393, 3)

In [ ]:
##### 폴더 구조 #####
## train
##  ㄴ 카리나
##  ㄴ 마동석
##  ㄴ 유재석
## val
##  ㄴ 카리나
##  ㄴ 마동석
##  ㄴ 유재석
## test
##  ㄴ 카리나
##  ㄴ 마동석
##  ㄴ 유재석


In [44]:
# 파일 옮기기
from pathlib import Path 
import shutil
import os

train_cnt = 70
val_cnt = 10
test_cnt = 20

path = Path("./celebrity")
celeb_list = ["카리나", "마동석", "유재석"]
target_folder = "train"
for celeb in celeb_list:
    celeb_path = path / celeb 
    cnt = 0
    for file in celeb_path.iterdir():
        # print(copy_file_name)
        sub_folder = path / target_folder / celeb
        if not sub_folder.exists():
            os.mkdir(sub_folder)
        shutil.copy(file, sub_folder)  # 복사하는 방법
        print(f"{target_folder} {cnt} 복사 완료! {file.name}")

        cnt += 1
        if cnt <= train_cnt:
            target_folder = "train"
        elif cnt <= train_cnt + val_cnt:
            target_folder = "val"  
        else:
            target_folder = "test"

train 0 복사 완료! Image_1.jpg
train 1 복사 완료! Image_10.jpg
train 2 복사 완료! Image_100.jpg
train 3 복사 완료! Image_11.jpg
train 4 복사 완료! Image_12.jpg
train 5 복사 완료! Image_13.JPG
train 6 복사 완료! Image_14.jpg
train 7 복사 완료! Image_15.jpg
train 8 복사 완료! Image_16.jpg
train 9 복사 완료! Image_17.jpg
train 10 복사 완료! Image_18.jpg
train 11 복사 완료! Image_19.jpg
train 12 복사 완료! Image_2.jpg
train 13 복사 완료! Image_20.jpg
train 14 복사 완료! Image_21.jpg
train 15 복사 완료! Image_22.jpg
train 16 복사 완료! Image_23.jpg
train 17 복사 완료! Image_24.jpg
train 18 복사 완료! Image_25.jpg
train 19 복사 완료! Image_26.png
train 20 복사 완료! Image_27.png
train 21 복사 완료! Image_28.png
train 22 복사 완료! Image_29.jpg
train 23 복사 완료! Image_3.jpg
train 24 복사 완료! Image_30.jpg
train 25 복사 완료! Image_31.jpg
train 26 복사 완료! Image_32.jpg
train 27 복사 완료! Image_33.jpg
train 28 복사 완료! Image_34.jpg
train 29 복사 완료! Image_35.jpg
train 30 복사 완료! Image_36.jpg
train 31 복사 완료! Image_37.jpg
train 32 복사 완료! Image_38.png
train 33 복사 완료! Image_39.jpg
train 34 복사 완료! Image_4.jp

## 2. 데이터 불러오기

In [146]:
from torchvision.datasets import ImageFolder 
import torchvision.transforms as transforms 

data_transform = transforms.Compose(
    [
        transforms.Resize((224, 224)),
        transforms.ToTensor(),
        transforms.Normalize((0.5,), (0.5,))
    ]
)

train_data = ImageFolder(root="./celebrity/train", transform=data_transform)
val_data = ImageFolder(root="./celebrity/val", transform=data_transform)
test_data = ImageFolder(root="./celebrity/test", transform=data_transform)
print(f"Train Data 개수: {len(train_data)}")
print(f"Validation Data 개수: {len(val_data)}")
print(f"Test Data 개수: {len(test_data)}")

Train Data 개수: 211
Validation Data 개수: 30
Test Data 개수: 59


In [147]:
celebrity_class = train_data.classes
print(celebrity_class)

['마동석', '유재석', '카리나']


In [ ]:
from torch.utils.data.dataloader import DataLoader

batch_size = 1

train_loader = DataLoader(train_data, batch_size=batch_size, shuffle=True)
val_loader = DataLoader(val_data, batch_size=batch_size, shuffle=True)
test_loader = DataLoader(test_data, batch_size=batch_size)

print(f"Train Data 배치 개수: {len(train_loader)}")
print(f"Validation Data 배치 개수: {len(val_loader)}")
print(f"Test Data 배치 개수: {len(test_loader)}")

Train Data 배치 개수: 211
Validation Data 배치 개수: 30
Test Data 배치 개수: 59


In [151]:
train_data[0][0].shape

torch.Size([3, 224, 224])

In [ ]:
for batch_data, batch_target in train_loader:
    print(batch_data.shape)
    print(batch_target)
    break

torch.Size([1, 3, 224, 224])
tensor([1])


## 3. 모델 가져오기

In [88]:
from torchvision.models.vgg import vgg16
from torchvision.models.resnet import resnet34

model = vgg16(weights='IMAGENET1K_V1')
# model = resnet34(pretrained=True)
model

VGG(
  (features): Sequential(
    (0): Conv2d(3, 64, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1))
    (1): ReLU(inplace=True)
    (2): Conv2d(64, 64, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1))
    (3): ReLU(inplace=True)
    (4): MaxPool2d(kernel_size=2, stride=2, padding=0, dilation=1, ceil_mode=False)
    (5): Conv2d(64, 128, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1))
    (6): ReLU(inplace=True)
    (7): Conv2d(128, 128, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1))
    (8): ReLU(inplace=True)
    (9): MaxPool2d(kernel_size=2, stride=2, padding=0, dilation=1, ceil_mode=False)
    (10): Conv2d(128, 256, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1))
    (11): ReLU(inplace=True)
    (12): Conv2d(256, 256, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1))
    (13): ReLU(inplace=True)
    (14): Conv2d(256, 256, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1))
    (15): ReLU(inplace=True)
    (16): MaxPool2d(kernel_size=2, stride=2, padding=0, dilation=1

In [89]:
import torch 
from torchsummary import summary 

device = "cuda" if torch.cuda.is_available() else "cpu"
model = model.to(device)

summary(model, (3, 224, 224))

----------------------------------------------------------------
        Layer (type)               Output Shape         Param #
            Conv2d-1         [-1, 64, 224, 224]           1,792
              ReLU-2         [-1, 64, 224, 224]               0
            Conv2d-3         [-1, 64, 224, 224]          36,928
              ReLU-4         [-1, 64, 224, 224]               0
         MaxPool2d-5         [-1, 64, 112, 112]               0
            Conv2d-6        [-1, 128, 112, 112]          73,856
              ReLU-7        [-1, 128, 112, 112]               0
            Conv2d-8        [-1, 128, 112, 112]         147,584
              ReLU-9        [-1, 128, 112, 112]               0
        MaxPool2d-10          [-1, 128, 56, 56]               0
           Conv2d-11          [-1, 256, 56, 56]         295,168
             ReLU-12          [-1, 256, 56, 56]               0
           Conv2d-13          [-1, 256, 56, 56]         590,080
             ReLU-14          [-1, 256,

In [90]:
model.classifier

Sequential(
  (0): Linear(in_features=25088, out_features=4096, bias=True)
  (1): ReLU(inplace=True)
  (2): Dropout(p=0.5, inplace=False)
  (3): Linear(in_features=4096, out_features=4096, bias=True)
  (4): ReLU(inplace=True)
  (5): Dropout(p=0.5, inplace=False)
  (6): Linear(in_features=4096, out_features=1000, bias=True)
)

## 4. 모델 수정

In [ ]:
# Freeze 기능
for param in model.parameters():
    param.requires_grad = False

In [92]:
import torch.nn as nn 

myclassifier = nn.Sequential(
    nn.Linear(in_features=25088, out_features=4096, bias=True),
    nn.ReLU(),
    nn.Dropout(),
    nn.Linear(in_features=4096, out_features=4096, bias=True),
    nn.ReLU(),
    nn.Linear(in_features=4096, out_features=3, bias=True)
)

In [93]:
model.classifier = myclassifier

In [94]:
model = model.to(device)

## 5. 학습

In [95]:
from torch.utils.tensorboard import SummaryWriter
import time

log_dir = f"runs/mydata/{time.strftime('%Y%m%d-%H%M%S')}"
writer = SummaryWriter(log_dir)

In [ ]:
import torch.optim as optim 

epochs = 200
batch_size = 1
criterion = nn.CrossEntropyLoss()
optimizer = optim.Adam(model.parameters(), lr=1e-3)

loss_history = {"train": [], "val": []}
acc_history = {"train": [], "val": []}

best_loss_val = float('inf')
patience = 10
patience_cnt = 0

for epoch in range(epochs):
    #### train ####
    model.train() 

    loss_train = 0.0
    corr_train = 0
    for data, target in train_loader:
        # GPU 보내기
        data = data.to(device)
        target = target.to(device)

        # 학습
        optimizer.zero_grad()
        yhat = model(data)
        loss = criterion(yhat, target)
        loss.backward()
        optimizer.step()

        # loss, correct 계산
        loss_train += loss.item() * batch_size
        corr_train += (yhat.argmax(dim=1) == target).sum().item()

    # loss, acc 저장
    loss_history["train"].append(loss_train / len(train_data))
    acc_history["train"].append(corr_train / len(train_data))


    #### validation ####
    model.eval()

    loss_val = 0.0
    corr_val = 0
    for data, target in val_loader:
        # GPU 보내기
        data = data.to(device)
        target = target.to(device)

        # 예측
        with torch.no_grad():
            pred = model(data)
            loss_val += criterion(pred, target).item() * batch_size
            corr_val += (pred.argmax(dim=1) == target).sum().item()

    # loss, acc 저장
    loss_history["val"].append(loss_val / len(val_data))
    acc_history["val"].append(corr_val / len(val_data))

    writer.add_scalars(
        "Loss", 
        {"Train": loss_train / len(train_data), "Validation": loss_val / len(val_data)}, 
        epoch
    )
    writer.add_scalars(
        "Accuracy", 
        {"Train": corr_train / len(train_data), "Validation": corr_val / len(val_data)}, 
        epoch
    )

    # Early Stopping
    if loss_val < best_loss_val:
        best_loss_val = loss_val
        torch.save(model.state_dict(), "celebrity.pth")
    else:
        patience_cnt += 1
        if patience_cnt == patience:
            print("Early Stopping!")
            break
    
    # 출력
    if epoch % 1 == 0:
        print(f"Epoch: {epoch}, Train Loss: {loss_train / len(train_data):.4f}, Train acc: {corr_train / len(train_data):.4f} Validation Loss: {loss_val / len(val_loader):.4f}, Validation acc: {corr_val / len(val_data):.4f}")

Epoch: 0, Train Loss: 4.3392, Train acc: 0.5450 Validation Loss: 8.4124, Validation acc: 0.5667
Epoch: 1, Train Loss: 2.9036, Train acc: 0.8341 Validation Loss: 24.8028, Validation acc: 0.4333
Epoch: 2, Train Loss: 2.2670, Train acc: 0.8768 Validation Loss: 1.3495, Validation acc: 0.8333
Epoch: 3, Train Loss: 2.9060, Train acc: 0.9005 Validation Loss: 6.8243, Validation acc: 0.7333
Epoch: 4, Train Loss: 2.2270, Train acc: 0.9194 Validation Loss: 4.6901, Validation acc: 0.8000
Epoch: 5, Train Loss: 2.8022, Train acc: 0.9242 Validation Loss: 1.5759, Validation acc: 0.9333
Epoch: 6, Train Loss: 2.4104, Train acc: 0.9242 Validation Loss: 18.6781, Validation acc: 0.7667
Epoch: 7, Train Loss: 1.0690, Train acc: 0.9763 Validation Loss: 6.2226, Validation acc: 0.8333
Epoch: 8, Train Loss: 2.1034, Train acc: 0.9621 Validation Loss: 19.9300, Validation acc: 0.8000
Epoch: 9, Train Loss: 5.9928, Train acc: 0.9431 Validation Loss: 9.0665, Validation acc: 0.9333
Epoch: 10, Train Loss: 9.1275, Train 

In [71]:
writer.close()

## 6. 예측

### 성능 평가

In [155]:
from torchvision.models.vgg import vgg16
from torchvision.models.resnet import resnet34
import torch.nn as nn 

model = vgg16(weights='IMAGENET1K_V1')

myclassifier = nn.Sequential(
    nn.Linear(in_features=25088, out_features=4096, bias=True),
    nn.ReLU(),
    nn.Dropout(),
    nn.Linear(in_features=4096, out_features=4096, bias=True),
    nn.ReLU(),
    nn.Linear(in_features=4096, out_features=3, bias=True)
)

model.classifier = myclassifier

In [156]:
# 저장된 모델 불러오기
model.load_state_dict(torch.load("celebrity.pth"))

<All keys matched successfully>

In [157]:
# 예측 -- 이 모델의 성능을 평가하기 위한 코드
device = "cuda" if torch.cuda.is_available() else "cpu"
model = model.to(device)

model.eval()
corr_test = 0
for data, target in test_loader:
    # GPU 보내기
    data = data.to(device)
    target = target.to(device)

    # 예측
    with torch.no_grad():
        pred = model(data)
        corr_test += (pred.argmax(dim=1) == target).sum().item()

print(corr_test / len(test_data))

0.6779661016949152


### 새로운 데이터 예측

In [123]:
# 모델 불러오기
import torch 

model = vgg16(weights='IMAGENET1K_V1')

myclassifier = nn.Sequential(
    nn.Linear(in_features=25088, out_features=4096, bias=True),
    nn.ReLU(),
    nn.Dropout(),
    nn.Linear(in_features=4096, out_features=4096, bias=True),
    nn.ReLU(),
    nn.Linear(in_features=4096, out_features=3, bias=True)
)

model.classifier = myclassifier

model.load_state_dict(torch.load("celebrity.pth"))

<All keys matched successfully>

In [ ]:
new_image_path = "./마동석_테스트.jpg"

from PIL import Image 

image = Image.open(new_image_path).convert("RGB")
image_transform = data_transform(image).unsqueeze(0)
print(image_transform.shape)

torch.Size([1, 3, 224, 224])


In [158]:
train_data.classes

['마동석', '유재석', '카리나']

In [160]:
pred = model(image_transform.to(device))
pred_idx = pred.argmax(dim=1).item()
celebrity_class[pred_idx]

'마동석'